In [1]:
###-- Google Driveに同期 --###
from google.colab import drive
drive.mount('/content/drive')

In [2]:
###------------------------------###
###           入力項目            ###
###------------------------------###
##-- CSVファイル名（訓練データ、テストデータ）
train_filename, test_filename = "train_data.csv", "test_data.csv"

##-- Data PATH
PATH_data = "drive/My Drive/jupyter/ProbSpace/YouTube/data/"
##-- Data Submit
PATH_submit = "drive/My Drive/jupyter/ProbSpace/YouTube/submit/"

##-- Rondom state
random_state = 900

In [2]:
###-------------------------------------------###
###        必要なライブラリの読み込み             ###
###-------------------------------------------###
##-- Pandas(CSVファイルの読み込み/処理)
import pandas as pd
from pandas import Series, DataFrame
pd.set_option('max_columns', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('max_rows', 300)
pd.set_option('display.max_rows', 300)
##-- Numpy(数値計算用の配列)
import numpy as np
##-- Matplotlib(グラフの描画)
import matplotlib.pylab as plt
import matplotlib.cm as cm # グラデーション
import seaborn as sns
##-- Scikit-learn(重回帰処理)
import sklearn  #-- print(sklearn.__version__)

##-- Warning非表示
import warnings
warnings.filterwarnings('ignore')

plt.style.use('bmh')
from itertools import cycle
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [4]:
###---------------------------------------------###
###        CSVファイルからプロセスを読み込む         ###
###---------------------------------------------###
f = pd.read_csv(PATH_data+"/"+train_filename, encoding="utf-8")
g = pd.read_csv(PATH_data+"/"+test_filename, encoding="utf-8")

##-- 対数をとっている
f["y"] = np.log(f["y"])

f.head(5)
# f.dtypes

,id,video_id,title,publishedAt,channelId,channelTitle,categoryId,collection_date,tags,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,y
0,1,GDtyztIThRQ,[12] BGM Inazuma Eleven 3 - ~ライオコツト ダンジョン~,2011-01-09T05:50:33.000Z,UCQaNYC3dNvH8FqrEyK7hTJw,DjangoShiny,20,20.01.02,Inazuma|Eleven|Super|Once|bgm|ost|イナズマイレブン|Kyo...,114,0,7,https://i.ytimg.com/vi/GDtyztIThRQ/default.jpg,False,False,~ライオコツト ダンジョン~Inazuma Eleven 3 BGM Complete (R...,10.282917
1,2,m4H9s3GtTlQ,ねごと - メルシールー [Official Music Video],2012-07-23T03:00:09.000Z,UChMWDi-HBm5aS3jyRSaAWUA,ねごと Official Channel,10,20.08.02,ねごと|ネゴト|メルシールー|Re:myend|リマインド|Lightdentity|ライデ...,2885,50,111,https://i.ytimg.com/vi/m4H9s3GtTlQ/default.jpg,False,False,http://www.negoto.com/全員平成生まれ、蒼山幸子（Vo＆Key)、沙田瑞...,13.501183
2,3,z19zYZuLuEU,VF3tb 闇よだれvsちび太 (SEGA),2007-07-26T13:54:09.000Z,UCBdcyoZSt5HBLd_n6we-xIg,siropai,24,20.14.01,VF3|VF4|VF5|ちび太|闇よだれ|chibita|virtuafighter|seg...,133,17,14,https://i.ytimg.com/vi/z19zYZuLuEU/default.jpg,False,False,Beat-tribe cup finalhttp://ameblo.jp/siropai/,11.298085
3,4,pmcIOsL7s98,free frosty weekend!,2005-05-15T02:38:43.000Z,UC7K5am1UAQEsCRhzXpi9i1g,Jones4Carrie,22,19.22.12,frosty,287,51,173,https://i.ytimg.com/vi/pmcIOsL7s98/default.jpg,False,False,I look so bad but look at me!,10.458120
4,5,ZuQgsTcuM-4,トップ・オブ・ザ・ワールド,2007-09-09T09:52:47.000Z,UCTW1um4R-QWa8iIfITGvlZQ,Tatsuya Maruyama,10,20.08.01,ギター|guitar|南澤大介|トップオブザワールド|トップ|オブ|ワールド|カーペンターズ...,178,6,17,https://i.ytimg.com/vi/ZuQgsTcuM-4/default.jpg,False,False,ソロギターのしらべより「トップオブザワールド」です。クラシックギターで弾いてます。Offic...,12.059468


前処理

In [5]:
mean_ = f[["categoryId", "y"]].groupby("categoryId").mean().reset_index().rename({"y":"mean"}, axis=1)
max_ = f[["categoryId", "y"]].groupby("categoryId").max().reset_index().rename({"y":"max"}, axis=1)
min_ = f[["categoryId", "y"]].groupby("categoryId").min().reset_index().rename({"y":"min"}, axis=1)
std_ = f[["categoryId", "y"]].groupby("categoryId").std().reset_index().rename({"y":"std"}, axis=1)
count_ = f[["categoryId", "y"]].groupby("categoryId").count().reset_index().rename({"y":"count"}, axis=1)
q1_ = f[["categoryId", "y"]].groupby("categoryId").quantile(0.1).reset_index().rename({"y":"q1"}, axis=1)
q25_ = f[["categoryId", "y"]].groupby("categoryId").quantile(0.25).reset_index().rename({"y":"q25"}, axis=1)
q5_ = f[["categoryId", "y"]].groupby("categoryId").quantile(0.5).reset_index().rename({"y":"q5"}, axis=1)
q75_ = f[["categoryId", "y"]].groupby("categoryId").quantile(0.75).reset_index().rename({"y":"q75"}, axis=1)
q9_ = f[["categoryId", "y"]].groupby("categoryId").quantile(0.9).reset_index().rename({"y":"q9"}, axis=1)


def is_japanese(string):
    for ch in string:
        try:
            name = unicodedata.name(ch) 
            if "CJK UNIFIED" in name \
            or "HIRAGANA" in name \
            or "KATAKANA" in name:
                return True
        except:
          continue
    return False

Y = f["y"]
del f["y"]

##-- Dataを結合
data = pd.concat([f, g])
print(data.shape)  # 19720 + 29582 = 49302

data["tags"].fillna("[none]", inplace=True)
tagdic = dict(pd.Series("|".join(list(data["tags"])).split("|")).value_counts().sort_values())



(49302, 16)


In [6]:
##-- 正規表現 https://docs.python.org/3/library/re.html
import re

def bool_to_int(df):
  df["comments_disabled"] = df["comments_disabled"].astype(np.int16)
  df["ratings_disabled"] = df["ratings_disabled"].astype(np.int16)
  return df

def create_features(df):
  ##--  like dislike comment
  df["likes2"] = df["likes"]**2
  df["loglikes"] = np.log(df["likes"]+1)
  df["dislikes2"] = df["dislikes"]**2
  df["logdislikes"] = np.log(df["dislikes"]+1)
  df["logcomment_count"] = np.log(df["comment_count"]+1)
  df["sqrtlikes"] = np.sqrt(df["likes"])
  df["like_dislike_ratio"] = df["likes"]/(df["dislikes"]+1)
  df["comments_like_ratio"] = df["comment_count"]/(df["likes"]+1)
  df["comments_dislike_ratio"] = df["comment_count"]/(df["dislikes"]+1)

  ##-- likes comments diable
  df["likes_com"] = df["likes"] * df["comments_disabled"]
  df["dislikes_com"] = df["dislikes"] * df["comments_disabled"]
  df["comments_likes"] = df["comment_count"] * df["ratings_disabled"]

  ##-- tags
  df["num_tags"] = df["tags"].astype(str).apply(lambda x: len(x.split("|")))
  df["length_tags"] = df["tags"].astype(str).apply(lambda x: len(x))
  df["tags_point"] = df["tags"].apply(lambda tags: sum([tagdic[tag] for tag in tags.split("|")]))
  df["count_en_tag"] = df["tags"].apply(lambda x: sum([bool(re.search(r'[a-zA-Z0-9]', x_)) for x_ in x.split("|")]))
  df["count_ja_tag"] = df["tags"].apply(lambda x: sum([is_japanese(x_) for x_ in x.split("|")]))

  ##-- publishedAt
  df["publishedAt"] = pd.to_datetime(df["publishedAt"], utc=True)
  df["publishedAt_year"] = df["publishedAt"].apply(lambda x: x.year)
  df["publishedAt_month"] = df["publishedAt"].apply(lambda x: x.month)
  df["publishedAt_day"] = df["publishedAt"].apply(lambda x: x.day)
  df["publishedAt_hour"] = df["publishedAt"].apply(lambda x: x.hour)
  df["publishedAt_minute"] = df["publishedAt"].apply(lambda x: x.minute)
  df["publishedAt_second"] = df["publishedAt"].apply(lambda x: x.second)
  df["publishedAt_dayofweek"] = df["publishedAt"].apply(lambda x: x.dayofweek)

  ##-- collection_date
  df["collection_date_year"] = df["collection_date"].apply(lambda x: int(x[0:2]))
  ##-- 20 --> 2020年に直す
  df["collection_date_year"] = df["collection_date_year"] + 2000
  ##--
  df["collection_date_month"] = df["collection_date"].apply(lambda x: int(x[3:5]))
  df["collection_date_day"] = df["collection_date"].apply(lambda x: int(x[6:8]))
  df["collection_date"] = pd.to_datetime("20"+df["collection_date"], format="%Y.%d.%m", utc=True)

  ##-- delta
  df["delta"] = (df["collection_date"] - df["publishedAt"]).apply(lambda x: x.days)
  df["logdelta"] = np.log(df["delta"])
  df["sqrtdelta"] = np.sqrt(df["delta"])
  df["published_delta"] = (df["publishedAt"] - df["publishedAt"].min()).apply(lambda x: x.days)
  df["collection_delta"] = (df["collection_date"] - df["collection_date"].min()).apply(lambda x: x.days)

  df["description"].fillna(" ", inplace=True)
  df["ishttp_in_dis"] = df["description"].apply(lambda x: x.lower().count("http"))
  df["len_description"] = df["description"].apply(lambda x: len(x))

  df["title"].fillna(" ", inplace=True)
  df["len_title"] = df["title"].apply(lambda x: len(x))

  ##-- is japanese
  df["isJa_title"] = df["title"].apply(lambda x: is_japanese(x))
  df["isJa_tags"] = df["tags"].apply(lambda x: is_japanese(x))
  df["isJa_description"] = df["description"].apply(lambda x: is_japanese(x))

  ##-- is englosh
  df["onEn_tags"] = df["tags"].apply(lambda x: x.encode('utf-8').isalnum())
  df["onEn_description"] = df["description"].apply(lambda x: x.encode('utf-8').isalnum())

  ##-- cotain englosh
  df["conEn_title"] = df["title"].apply(lambda x: len(re.findall(r'[a-zA-Z0-9]', x.lower())))
  df["conEn_tags"] = df["tags"].apply(lambda x: len(re.findall(r'[a-zA-Z0-9]', x.lower())))
  df["conEn_description"] = df["description"].apply(lambda x: len(re.findall(r'[a-zA-Z0-9]', x.lower())))


  
  df = df.merge(mean_, how='left', on=["categoryId"])
  df = df.merge(max_, how='left', on=["categoryId"])
  df = df.merge(min_, how='left', on=["categoryId"])
  df = df.merge(std_, how='left', on=["categoryId"])
  df = df.merge(q1_, how='left', on=["categoryId"])
  df = df.merge(q25_, how='left', on=["categoryId"])
  df = df.merge(q5_, how='left', on=["categoryId"])
  df = df.merge(q75_, how='left', on=["categoryId"])
  df = df.merge(q9_, how='left', on=["categoryId"])

  # 出現頻度
  for col in ["categoryId", "channelTitle"]:
    freq = df[col].value_counts()
    df["freq_"+col] = df[col].map(freq)

  return df

In [7]:
###-- 特徴量を一挙作成
data = bool_to_int(data)
data = create_features(data)

In [8]:
###-- 特徴量（公開されている時間（hour））, Google検索値を使用
data["period_hour"] = 8760*(data["collection_date_year"] - data["publishedAt_year"]) \
                      + 730*(data["collection_date_month"] - data["publishedAt_month"]) \
                      + 24*(data["collection_date_day"] - data["publishedAt_day"])

In [9]:
del data["channelId"]
del data["video_id"]
del data["title"]
del data["description"]
del data["thumbnail_link"]
del data["channelTitle"]
del data["tags"]
del data["publishedAt"]
del data["collection_date"]
del data["id"]

In [10]:
##-- Feature importance 0 のcolumnsを削除
del data["logdislikes"]
del data["dislikes2"]
del data["loglikes"]
del data["count_ja_tag"]
del data["logcomment_count"]
del data["likes2"]
del data["isJa_description"]
del data["isJa_tags"]
del data["isJa_title"]
del data["sqrtlikes"]
del data["sqrtdelta"]
del data["logdelta"]

In [11]:
data.head()
# data.describe()

,categoryId,likes,dislikes,comment_count,comments_disabled,ratings_disabled,like_dislike_ratio,comments_like_ratio,comments_dislike_ratio,likes_com,dislikes_com,comments_likes,num_tags,length_tags,tags_point,count_en_tag,publishedAt_year,publishedAt_month,publishedAt_day,publishedAt_hour,publishedAt_minute,publishedAt_second,publishedAt_dayofweek,collection_date_year,collection_date_month,collection_date_day,delta,published_delta,collection_delta,ishttp_in_dis,len_description,len_title,onEn_tags,onEn_description,conEn_title,conEn_tags,conEn_description,mean,max,min,std,q1,q25,q5,q75,q9,freq_categoryId,freq_channelTitle,period_hour
0,20,114,0,7,0,0,114.000000,0.060870,7.000000,0,0,0,48,315,5434,47,2011,1,9,5,50,33,6,2020,1,2,3309,2086,41,0,61,42,False,False,19,258,38,12.746174,16.787970,3.871201,1.794815,10.331027,11.707977,12.980648,13.965945,14.772192,2275,1,78672
1,10,2885,50,111,0,0,56.568627,0.038462,2.176471,0,0,0,19,129,76,5,2012,7,23,3,0,9,0,2020,8,2,2755,2646,48,1,261,35,False,False,18,38,54,13.155115,20.987221,1.791759,2.345023,10.297022,11.692096,13.236976,14.651161,15.922321,17253,5,70306
2,24,133,17,14,0,0,7.388889,0.104478,0.777778,0,0,0,9,52,49,6,2007,7,26,13,54,9,3,2020,14,1,4554,823,23,1,45,22,False,False,11,33,36,12.621979,19.241236,1.098612,2.240565,9.784028,11.271160,12.789547,14.163681,15.277146,6873,1,118390
3,22,287,51,173,0,0,5.519231,0.600694,3.326923,0,0,0,1,6,2,1,2005,5,15,2,38,43,6,2019,22,12,5333,20,0,0,30,20,True,False,17,6,21,11.934383,17.548095,2.079442,2.559307,8.288986,10.549596,12.196820,13.755893,15.006330,2207,10,134978
4,10,178,6,17,0,0,25.428571,0.094972,2.428571,0,0,0,12,83,850,2,2007,9,9,9,52,47,6,2020,8,1,4503,868,17,2,138,13,False,False,0,10,79,13.155115,20.987221,1.791759,2.345023,10.297022,11.692096,13.236976,14.651161,15.922321,17253,2,112958


訓練・検証データセットの準備

In [12]:
##-- 数値変数
nume_cols = [\
              #-- "***"(int64)
              "likes",\
              #-- "***"(int64)
              "dislikes",\
              #-- "***"(int64)
              "comment_count",\
              "like_dislike_ratio",\
              "comments_like_ratio",\
              "comments_dislike_ratio",\
              ###--------------------------
              ###--- 特徴量減らさないと計算重くて回らない
              ###--------------------------
              # "num_tags",\
              # "tags_point",\
              # "count_en_tag",\
              # "len_description",\
              # "len_title",\
              # "conEn_title",\
              # "conEn_tags",\
              # "conEn_description",\
              # #-- "***"(float64)
              # "length_tags",\
              # #-- "***"(float64)
              # "period_hour",\
             ]
##-- カテゴリカル変数
cat_cols = [\
              #-- "***"(int64)
              "categoryId",\
              "freq_categoryId",\
              "freq_channelTitle",\
              # "isJa_title",\
              # "isJa_tags",\
              # "isJa_description",\
              "onEn_tags",\
              "onEn_description",\
              "ishttp_in_dis",\
            ]

In [13]:
##-- 交互作用の作成
def interaction(df, nume_cols):
  k = 0
  for i in range(len(nume_cols)):
      for j in range(len(nume_cols)):
          if i == j:
              pass
          else:
              k += 1
              ##-- 積
              name_seki = "feature_seki" + str(k)
              df[name_seki] = df[nume_cols[i]]*df[nume_cols[j]]
              ##-- 商
              name_shou = "feature_shou" + str(k)
              df[name_shou] = df[nume_cols[i]]*df[nume_cols[j]]
              ##-- 差
              name_sa = "feature_sa" + str(k)
              df[name_sa] = df[nume_cols[i]]*df[nume_cols[j]]
  return df
data = interaction(data, nume_cols)
# X.head()

In [14]:
data.head()

,categoryId,likes,dislikes,comment_count,comments_disabled,ratings_disabled,like_dislike_ratio,comments_like_ratio,comments_dislike_ratio,likes_com,dislikes_com,comments_likes,num_tags,length_tags,tags_point,count_en_tag,publishedAt_year,publishedAt_month,publishedAt_day,publishedAt_hour,publishedAt_minute,publishedAt_second,publishedAt_dayofweek,collection_date_year,collection_date_month,collection_date_day,delta,published_delta,collection_delta,ishttp_in_dis,len_description,len_title,onEn_tags,onEn_description,conEn_title,conEn_tags,conEn_description,mean,max,min,std,q1,q25,q5,q75,q9,freq_categoryId,freq_channelTitle,period_hour,feature_seki1,feature_shou1,feature_sa1,feature_seki2,feature_shou2,feature_sa2,feature_seki3,feature_shou3,feature_sa3,feature_seki4,feature_shou4,feature_sa4,feature_seki5,feature_shou5,feature_sa5,feature_seki6,feature_shou6,feature_sa6,feature_seki7,feature_shou7,feature_sa7,feature_seki8,feature_shou8,feature_sa8,feature_seki9,feature_shou9,feature_sa9,feature_seki10,feature_shou10,feature_sa10,feature_seki11,feature_shou11,feature_sa11,feature_seki12,feature_shou12,feature_sa12,feature_seki13,feature_shou13,feature_sa13,feature_seki14,feature_shou14,feature_sa14,feature_seki15,feature_shou15,feature_sa15,feature_seki16,feature_shou16,feature_sa16,feature_seki17,feature_shou17,feature_sa17,feature_seki18,feature_shou18,feature_sa18,feature_seki19,feature_shou19,feature_sa19,feature_seki20,feature_shou20,feature_sa20,feature_seki21,feature_shou21,feature_sa21,feature_seki22,feature_shou22,feature_sa22,feature_seki23,feature_shou23,feature_sa23,feature_seki24,feature_shou24,feature_sa24,feature_seki25,feature_shou25,feature_sa25,feature_seki26,feature_shou26,feature_sa26,feature_seki27,feature_shou27,feature_sa27,feature_seki28,feature_shou28,feature_sa28,feature_seki29,feature_shou29,feature_sa29,feature_seki30,feature_shou30,feature_sa30
0,20,114,0,7,0,0,114.000000,0.060870,7.000000,0,0,0,48,315,5434,47,2011,1,9,5,50,33,6,2020,1,2,3309,2086,41,0,61,42,False,False,19,258,38,12.746174,16.787970,3.871201,1.794815,10.331027,11.707977,12.980648,13.965945,14.772192,2275,1,78672,0,0,0,798,798,798,12996.000000,12996.000000,12996.000000,6.939130,6.939130,6.939130,798.000000,798.000000,798.000000,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,798,798,798,0,0,0,798.000000,798.000000,798.000000,0.426087,0.426087,0.426087,49.000000,49.000000,49.000000,12996.000000,12996.000000,12996.000000,0.000000,0.000000,0.000000,798.000000,798.000000,798.000000,6.939130,6.939130,6.939130,798.000000,798.000000,798.000000,6.939130,6.939130,6.939130,0.000000,0.000000,0.000000,0.426087,0.426087,0.426087,6.939130,6.939130,6.939130,0.426087,0.426087,0.426087,798.000000,798.000000,798.000000,0.000000,0.000000,0.000000,49.000000,49.000000,49.000000,798.000000,798.000000,798.000000,0.426087,0.426087,0.426087
1,10,2885,50,111,0,0,56.568627,0.038462,2.176471,0,0,0,19,129,76,5,2012,7,23,3,0,9,0,2020,8,2,2755,2646,48,1,261,35,False,False,18,38,54,13.155115,20.987221,1.791759,2.345023,10.297022,11.692096,13.236976,14.651161,15.922321,17253,5,70306,144250,144250,144250,320235,320235,320235,163200.490196,163200.490196,163200.490196,110.961538,110.961538,110.961538,6279.117647,6279.117647,6279.117647,144250,144250,144250,5550,5550,5550,2828.431373,2828.431373,2828.431373,1.923077,1.923077,1.923077,108.823529,108.823529,108.823529,320235,320235,320235,5550,5550,5550,6279.117647,6279.117647,6279.117647,4.269231,4.269231,4.269231,241.588235,241.588235,241.588235,163200.490196,163200.490196,163200.490196,2828.431373,2828.431373,2828.431373,6279.117647,6279.117647,6279.117647,2.175716,2.175716,2.175716,123.119954,123.119954,123.119954,110.961538,110.961538,110.961538,1.923077,1.923077,1.923077,4.269231,4.269231,4.269231,2.175716,2.175716,2.175716,0.083710,0.083710,0.083710,6279.117647,6279.117647,6279.117647,108.823529,108.823529,108.823529,241.588235,241.588235,241.588235,123.119954,123.119954,12

In [15]:
##-- Label Encoding for categorical variable
from sklearn import preprocessing

for name in cat_cols:
  ##-- 学習データに基づいて定義する
  le = preprocessing.LabelEncoder()
  ##-- 訓練とテスト用の併せたカテゴリーにする
  # X_fit = X[name].append(X_pre[name])
  # le.fit(X_fit)
  le.fit(data[name])

  data[name] = le.transform(data[name])

data.head()

,categoryId,likes,dislikes,comment_count,comments_disabled,ratings_disabled,like_dislike_ratio,comments_like_ratio,comments_dislike_ratio,likes_com,dislikes_com,comments_likes,num_tags,length_tags,tags_point,count_en_tag,publishedAt_year,publishedAt_month,publishedAt_day,publishedAt_hour,publishedAt_minute,publishedAt_second,publishedAt_dayofweek,collection_date_year,collection_date_month,collection_date_day,delta,published_delta,collection_delta,ishttp_in_dis,len_description,len_title,onEn_tags,onEn_description,conEn_title,conEn_tags,conEn_description,mean,max,min,std,q1,q25,q5,q75,q9,freq_categoryId,freq_channelTitle,period_hour,feature_seki1,feature_shou1,feature_sa1,feature_seki2,feature_shou2,feature_sa2,feature_seki3,feature_shou3,feature_sa3,feature_seki4,feature_shou4,feature_sa4,feature_seki5,feature_shou5,feature_sa5,feature_seki6,feature_shou6,feature_sa6,feature_seki7,feature_shou7,feature_sa7,feature_seki8,feature_shou8,feature_sa8,feature_seki9,feature_shou9,feature_sa9,feature_seki10,feature_shou10,feature_sa10,feature_seki11,feature_shou11,feature_sa11,feature_seki12,feature_shou12,feature_sa12,feature_seki13,feature_shou13,feature_sa13,feature_seki14,feature_shou14,feature_sa14,feature_seki15,feature_shou15,feature_sa15,feature_seki16,feature_shou16,feature_sa16,feature_seki17,feature_shou17,feature_sa17,feature_seki18,feature_shou18,feature_sa18,feature_seki19,feature_shou19,feature_sa19,feature_seki20,feature_shou20,feature_sa20,feature_seki21,feature_shou21,feature_sa21,feature_seki22,feature_shou22,feature_sa22,feature_seki23,feature_shou23,feature_sa23,feature_seki24,feature_shou24,feature_sa24,feature_seki25,feature_shou25,feature_sa25,feature_seki26,feature_shou26,feature_sa26,feature_seki27,feature_shou27,feature_sa27,feature_seki28,feature_shou28,feature_sa28,feature_seki29,feature_shou29,feature_sa29,feature_seki30,feature_shou30,feature_sa30
0,6,114,0,7,0,0,114.000000,0.060870,7.000000,0,0,0,48,315,5434,47,2011,1,9,5,50,33,6,2020,1,2,3309,2086,41,0,61,42,0,0,19,258,38,12.746174,16.787970,3.871201,1.794815,10.331027,11.707977,12.980648,13.965945,14.772192,12,0,78672,0,0,0,798,798,798,12996.000000,12996.000000,12996.000000,6.939130,6.939130,6.939130,798.000000,798.000000,798.000000,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,798,798,798,0,0,0,798.000000,798.000000,798.000000,0.426087,0.426087,0.426087,49.000000,49.000000,49.000000,12996.000000,12996.000000,12996.000000,0.000000,0.000000,0.000000,798.000000,798.000000,798.000000,6.939130,6.939130,6.939130,798.000000,798.000000,798.000000,6.939130,6.939130,6.939130,0.000000,0.000000,0.000000,0.426087,0.426087,0.426087,6.939130,6.939130,6.939130,0.426087,0.426087,0.426087,798.000000,798.000000,798.000000,0.000000,0.000000,0.000000,49.000000,49.000000,49.000000,798.000000,798.000000,798.000000,0.426087,0.426087,0.426087
1,2,2885,50,111,0,0,56.568627,0.038462,2.176471,0,0,0,19,129,76,5,2012,7,23,3,0,9,0,2020,8,2,2755,2646,48,1,261,35,0,0,18,38,54,13.155115,20.987221,1.791759,2.345023,10.297022,11.692096,13.236976,14.651161,15.922321,17,4,70306,144250,144250,144250,320235,320235,320235,163200.490196,163200.490196,163200.490196,110.961538,110.961538,110.961538,6279.117647,6279.117647,6279.117647,144250,144250,144250,5550,5550,5550,2828.431373,2828.431373,2828.431373,1.923077,1.923077,1.923077,108.823529,108.823529,108.823529,320235,320235,320235,5550,5550,5550,6279.117647,6279.117647,6279.117647,4.269231,4.269231,4.269231,241.588235,241.588235,241.588235,163200.490196,163200.490196,163200.490196,2828.431373,2828.431373,2828.431373,6279.117647,6279.117647,6279.117647,2.175716,2.175716,2.175716,123.119954,123.119954,123.119954,110.961538,110.961538,110.961538,1.923077,1.923077,1.923077,4.269231,4.269231,4.269231,2.175716,2.175716,2.175716,0.083710,0.083710,0.083710,6279.117647,6279.117647,6279.117647,108.823529,108.823529,108.823529,241.588235,241.588235,241.588235,123.119954,123.119954,123.119954,0.083710,0.083

In [16]:
###-- データセットのメモリ削減 --###
##-- Ref. https://www.kaggle.com/fabiendaniel/elo-world
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print('start size: {:5.2f} Mb'.format(data.memory_usage().sum() / 1024**2))
data = reduce_mem_usage(data)

start size: 52.10 Mb
Mem. usage decreased to 23.89 Mb (54.2% reduction)


In [17]:
##-- 説明変数のみ取り出す
# data = data[input_name]

##-- 訓練データ(X)とテストデータ(X_test)に再分割
X = data.iloc[:len(Y), :]
X_pre = data.iloc[len(Y):, :]

print(X.shape[0], X_pre.shape[0]) ##-- 19720 and 29582

19720 29582


In [3]:
###-- Import LightGBM and MultiClass logloss --###
import lightgbm as lgb
from sklearn.metrics import log_loss, mean_squared_log_error
###--  Set params  --###
params = {
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'rmse',
          'max_depth': 8,
          'num_threads': 4,
          'verbose': -1,
          #-- Optimized by Optuna
          'learning_rate': 0.07,
          'lambda_l1': 0.0057,
          'lambda_l2': 0.,
          'num_leaves': 222,
          'feature_fraction': 1.,
          'bagging_fraction': 0.93,
          'bagging_freq': 1,
          'min_child_samples': 28,
          #-- Added
          'seed': random_state,                                                                        
}
###-- データの8割を訓練に、残り2割を検証データに割り振る
rate_split = 0.8
X_train, Y_train = X.iloc[:int(len(Y)*rate_split), :], Y.iloc[:int(len(Y)*rate_split)]
X_val, Y_val = X.iloc[int(len(Y)*rate_split):, :], Y.iloc[int(len(Y)*rate_split):]
###-- Set dataset --###
train_data_set = lgb.Dataset(X_train, Y_train, categorical_feature=cat_cols, free_raw_data=False)
test_data_set = lgb.Dataset(X_val, Y_val, reference=train_data_set, categorical_feature=cat_cols)
###-- 全データ投入して学習 --###
models, loss_list = [], []

num_splits=4
from sklearn.model_selection import KFold
kf = KFold(n_splits=num_splits, shuffle=True, random_state=random_state)
##------------------------------------
for train_idx, val_idx in kf.split(X):
  ###-- Set the dataset --###
  X_kfold_train, Y_kfold_train = X.iloc[train_idx, :], Y.iloc[train_idx]
  X_kfold_val, Y_kfold_val = X.iloc[val_idx, :], Y.iloc[val_idx]

  ###-- Set dataset --###
  train_data_set = lgb.Dataset(X_kfold_train, Y_kfold_train, categorical_feature=cat_cols)
  test_data_set = lgb.Dataset(X_kfold_val, Y_kfold_val, reference=train_data_set)
  gbm = lgb.train(\
                  params,
                  train_data_set,
                  valid_names=['train', 'valid'],
                  valid_sets=test_data_set,
                  # early_stopping_rounds=20,  ##--default: 20
                  # verbose_eval=40,
                  # num_boost_round=100,
                  early_stopping_rounds=100,  ##--default: 20
                  verbose_eval=100,
                  num_boost_round=5000, #--Equaling to "n_estimators"
                  )
  models.append(gbm)

  Y_val_pre = gbm.predict(X_kfold_val)
  ##-- loss: RMSE  np.sqrt(mean_squared_log_error(np.exp(y_val), oof)) = 
  loss_rmsle = np.sqrt( mean_squared_log_error(np.exp(Y_val_pre), np.exp(Y_kfold_val)) )
  loss_list.append(loss_rmsle)
  ##-- RMSLE lossを出力
  print()
  print("RMSLE", loss_rmsle)
  print()


In [19]:
##-- Feature importanceの平均を各モデルごとにとる
def feature_importance(gbm):
  return pd.DataFrame({'Feature': X.columns,\
                       'importance':gbm.feature_importance(importance_type='gain')})\
                        .sort_values('importance', ascending=False)

feature, importance = [], []
for i in range(len(X.columns)):
  tem = 0.
  for j in range(len(models)):
  # for j in range(num_models):
    feature_imp = feature_importance(models[j])
    tem = tem + feature_imp["importance"][i]
  tem = tem / len(models)
  importance.append(tem)
  name = feature_imp["Feature"][i]
  feature.append(name)

##-- 寄与がゼロの"Feature"をpick up
f_imp = pd.DataFrame({'Feature': feature, 'importance': importance})
f_imp = f_imp[ f_imp["importance"] == 0. ]
f_imp["Feature"]

46     freq_categoryId
50       feature_shou1
51         feature_sa1
53       feature_shou2
54         feature_sa2
56       feature_shou3
57         feature_sa3
59       feature_shou4
60         feature_sa4
62       feature_shou5
63         feature_sa5
64       feature_seki6
65       feature_shou6
66         feature_sa6
68       feature_shou7
69         feature_sa7
71       feature_shou8
72         feature_sa8
74       feature_shou9
75         feature_sa9
77      feature_shou10
78        feature_sa10
79      feature_seki11
80      feature_shou11
81        feature_sa11
82      feature_seki12
83      feature_shou12
84        feature_sa12
85      feature_seki13
86      feature_shou13
87        feature_sa13
89      feature_shou14
90        feature_sa14
92      feature_shou15
93        feature_sa15
94      feature_seki16
95      feature_shou16
96        feature_sa16
97      feature_seki17
98      feature_shou17
99        feature_sa17
100     feature_seki18
101     feature_shou18
102       f

In [20]:
##-- Drop対象になったカテゴリカル変数を落としておく
##-- カテゴリカル変数
cat_cols = [\
              #-- "***"(int64)
              "categoryId",\
              "freq_categoryId",\
              "freq_channelTitle",\
              "onEn_tags",\
              "onEn_description",\
              "ishttp_in_dis",\
            ]

In [21]:
##-- 寄与がゼロの"Feature"をDrop
print("Before:", len(data.columns))
data = data.drop(columns=f_imp["Feature"])
print("Before:", len(data.columns))

Before: 139
Before: 62


In [22]:
##-- 訓練データ(X)とテストデータ(X_test)に再分割
X = data.iloc[:len(Y), :]
X_pre = data.iloc[len(Y):, :]

del data

print(X.shape[0], X_pre.shape[0]) ##-- 19720 and 29582

19720 29582


In [4]:
###-- Import Random Forest and MultiClass logloss --###
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss, mean_squared_log_error

###-- 全データ投入して学習 --###

###--  学習  --###
models, loss_list = [], []
num_models = 1

i = 0
for i in range(num_models):
  # params["seed"] = 99 + i
  random_state = random_state + 1
  print()
  print("Random seed:", random_state)
  print()
  ###-- KFold Cross validation --###
  num_splits=5
  from sklearn.model_selection import KFold
  kf = KFold(n_splits=num_splits, shuffle=True, random_state=random_state)
  ##----------------------------------------
  for train_idx, val_idx in kf.split(X):
    ###-- Set the dataset --###
    X_kfold_train, Y_kfold_train = X.iloc[train_idx, :], Y.iloc[train_idx]
    X_kfold_val, Y_kfold_val = X.iloc[val_idx, :], Y.iloc[val_idx]
    ##-- Random Forest Regression
    rf = RandomForestRegressor(
                                n_estimators = 1000,
                                criterion = 'mse', 
                                max_depth = None, #--None
                                min_samples_split = 2,
                                min_samples_leaf = 1,
                                bootstrap = True,
                                max_features = "auto",
                                random_state = random_state,
                                n_jobs = -1,
                             )
    rf.fit(X_kfold_train, Y_kfold_train)
    models.append(rf)

    Y_val_pre = rf.predict(X_kfold_val)
    ##-- loss: RMSE  np.sqrt(mean_squared_log_error(np.exp(y_val), oof)) = 
    loss_rmsle = np.sqrt( mean_squared_log_error(np.exp(Y_val_pre), np.exp(Y_kfold_val)) )
    loss_list.append(loss_rmsle)
    ##-- RMSLE lossを出力
    print()
    print("RMSLE", loss_rmsle)
    print()


In [25]:
# pd.DataFrame({'Feature': X_train.columns,\
#               'importance':rf.feature_importances_})\
#               .sort_values('importance', ascending=False)

##-- Feature importanceの平均を各モデルごとにとる
def feature_importance(rf):
  return pd.DataFrame({'Feature': X.columns,\
              'importance':rf.feature_importances_})\
              .sort_values('importance', ascending=False)

feature, importance = [], []
for i in range(len(X.columns)):
  tem = 0.
  for j in range(len(models)):
    feature_imp = feature_importance(models[j])
    tem = tem + feature_imp["importance"][i]
  tem = tem / len(models)
  importance.append(tem)
  name = feature_imp["Feature"][i]
  feature.append(name)

pd.DataFrame({'Feature': feature, 'importance': importance}).sort_values('importance', ascending=False)

,Feature,importance
48,feature_seki1,0.687223
5,ratings_disabled,0.066902
1,likes,0.034718
50,feature_seki3,0.015642
27,published_delta,0.014604
41,q1,0.013227
47,period_hour,0.012139
46,freq_channelTitle,0.011303
26,delta,0.011189
34,conEn_title,0.007196


train-average loss: 0.8003839551568694 \
validation loss: 0.789773304798804

In [26]:
def predict_ensemble(models, num_model, X_pre):
    Y_pre_list = []
    for i in range(num_model):
        Y_pre = models[i].predict(X_pre)
        Y_pre_list.append(Y_pre)

    Y_pre_list = np.array(Y_pre_list)

    Y_pre_submit_tem = Y_pre_list[0]
    for i in range(1, num_model):
        Y_pre_submit_tem += Y_pre_list[i]

    Y_pre_submit = Y_pre_submit_tem / float(num_model)

    return Y_pre_submit


###------------------------------------###
###        結果ファイルへの書き出し        ###
###------------------------------------###
from pathlib import Path
root = Path(PATH_submit)

##-- Prediction for test data
Y_pre_submit = np.exp(predict_ensemble(models, len(models), X_pre.fillna(X_pre.mean())))

submit = pd.DataFrame({'y': Y_pre_submit})
submit.index.name = 'id'
submit.index = submit.index + 1
submit.to_csv(root.joinpath("submission_RandomForest_5kfold_Seed"+str(random_state)+".csv"), index=True)